In [18]:
import random
import numpy as np
from  itertools import combinations


In [31]:
def resample(l):
    final = []
    for i in range(len(l)):
        final.append(random.choice(l))
    return final
def repeat_resample(sample_a, sample_b, num_iter = 1000):
    difference_in_means = []#keep track of the difference in heights for each experiment
    for i in range(num_iter):
        resample_a = resample(sample_a)
        resample_b = resample(sample_b)
        difference = np.mean(resample_a) - np.mean(resample_b)
        difference_in_means.append(difference)
    return difference_in_means

def get_p_value(s1, s2, resamps):
    if np.mean(s1) > np.mean(s2):
        return 1 - len([x for x in resamps if x > 0])/len(resamps)
    return 1 - len([x for x in resamps if x < 0])/len(resamps)



In [235]:
def comb_samples_to_pop(*args):
    all_ = []
    for i in args:
        all_.extend(i)
    return all_

def diff_from_pop(*args, num_iter = 1000):
    diffs = []
    for i in args:
        diffs.append([])
    pop = comb_samples_to_pop(*args)
    for i in range(num_iter):
        random.shuffle(pop)
        len_prev = 0
        for counter, j in enumerate(args):
            new_s = pop[len_prev:len(j) + len_prev]
            len_prev = len(j)
            diff = np.mean(j) - np.mean(new_s)
            diffs[counter].append(diff)
    return diffs

def eval_pops(diffs):
    r = []
    for i in diffs:
        p_value1 = len([x for x in i if x < 0])/len(i)
        p_value2 = len([x for x in i if x > 0])/len(i)
        if min(p_value1, p_value2) <= .01:
            r.append(True)
        else:
            r.append(False)
    return r

In [260]:
def test1():
    p_size = 100
    pop = [random.gauss(0,1) for x in range(p_size)]
    s1 = pop[0:20]
    s2 = pop[20:40]
    s3 = pop[40:60]
    s4 = pop[60:80]
    s5 = pop[80:100]
    print(np.mean(s1), np.mean(s2), np.mean(s3), np.mean(s4), np.mean(s5))
    combs = list(combinations([s1, s2, s3, s4, s5], 2))
    d = diff_from_pop(s1, s2, s3, s4, s5)
    f = True in eval_pops(d)

    for i in combs:
        resamps = repeat_resample(i[0], i[1])
        p_value = get_p_value(i[0], i[1], resamps)
        if p_value <= .05 and f:
            print(p_value)

test1()

0.29587145682986776 -0.3093684894212786 0.295545108587036 -0.6860850927296207 -0.4271987761976114
0.03300000000000003
0.0030000000000000027
0.018000000000000016
0.01200000000000001
0.0
0.006000000000000005


In [271]:
#want to test is combining sasples = ind test
def test2(num_iter = 1000):
    p_size = 100
    s1 = [random.gauss(0,1) for x in range(p_size)]
    s2 = [random.gauss(0,1) for x in range(p_size)]
    resamp = comb_samples_to_pop(s1, s2)
    for i in range(num_iter):
        random.shuffle(resamp)
        s1_ = resamp[:len(s1)]
        s2_ = resamp[len(s2):]

    

test2()